In [ ]:
# one of the main reviewer comments is that the viral prediction software we used (virsorter2) is out of date
# Instead, reviewers suggest we use genomad
# I have install genomad using mamba
# lets try it out

# First, copy over our virome and total metagenome assemblies to new dir
cp /home/dsbard/bee_phage/dls/5kb/virome/assemblies/5kb/* ./
cp cp /home/dsbard/bee_phage/dls/5kb/total_metagenome/assemblies/5kb/* ./

# run genomad
### total metagenome samples
#!/bin/bash
#SBATCH --job-name=Genomad
#SBATCH --nodes=1
#SBATCH -t 6:00:00
#SBATCH --ntasks=12
#SBATCH --output=/home/dsbard/bee_phage/reviewer_approach/genomad_out/log/log/%j.out
#SBATCH --partition=med

conda activate genomad

genomad \
end-to-end \
--cleanup \
--splits 8 \
/home/dsbard/bee_phage/reviewer_approach/assemblies/total_metagenomes/$1 \
/home/dsbard/bee_phage/reviewer_approach/assemblies/genomad_out/total_out \
/home/dsbard/genomad_db/genomad_db

# run from /home/dsbard/bee_phage/reviewer_approach/assemblies/total_metagenomes
for f in *.fasta
do
sbatch ../../scripts/genomad_total.sh $f
done

### Virome samples
#!/bin/bash
#SBATCH --job-name=Genomad
#SBATCH --nodes=1
#SBATCH -t 6:00:00
#SBATCH --ntasks=12
#SBATCH --output=/home/dsbard/bee_phage/reviewer_approach/genomad_out/log/log/%j.out
#SBATCH --partition=med

conda activate genomad

genomad \
end-to-end \
--cleanup \
--splits 8 \
/home/dsbard/bee_phage/reviewer_approach/assemblies/viromes/$1 \
/home/dsbard/bee_phage/reviewer_approach/genomad_out/virome_out \
/home/dsbard/genomad_db/genomad_db

# run from /home/dsbard/bee_phage/reviewer_approach/assemblies/viromes
for f in *.fasta
do
sbatch ../../scripts/genomad_virome.sh $f
done

In [ ]:
# place IDed phage into new dir
cd /home/dsbard/bee_phage/reviewer_approach/genomad_out
cd viral_seqs
mkdir viral_seqs/total viral_seqs/virome
for f in total_out/*summary/*virus.fna; do cp $f ./viral_seqs/total; done
for f in virome_out/*summary/*virus.fna; do cp $f ./viral_seqs/virome; done

# count how many phage seqs we got out per sample
## Total
cd /home/dsbard/bee_phage/reviewer_approach/genomad_out/viral_seqs/total

# Honey bee
for f in *MEL*.fna
do
sam=${f%%virsorter*}
n_phage=$(grep ">" $f | wc -l)
echo $sam,$n_phage
done >> honey_total_phage_counts.txt

# Bumble bee
for f in *IMP*.fna
do
sam=${f%%virsorter*}
n_phage=$(grep ">" $f | wc -l)
echo $sam,$n_phage
done >> bumble_total_phage_counts.txt

## virome
cd /home/dsbard/bee_phage/reviewer_approach/genomad_out/viral_seqs/virome

# Honey bee
for f in *MEL*.fna
do
sam=${f%%virsorter*}
n_phage=$(grep ">" $f | wc -l)
echo $sam,$n_phage
done >> honey_virome_phage_counts.txt

# Bumble bee
for f in *IMP*.fna
do
sam=${f%%virsorter*}
n_phage=$(grep ">" $f | wc -l)
echo $sam,$n_phage
done >> bumble_virome_phage_counts.txt

# count number of phage IDed by each sample geNomad
cd /home/dsbard/bee_phage/reviewer_approach/genomad_out
# virome samples
for f in virome_out/*summary/*virus.fna
do
    sample=${f%%_summary*}
    sample_2=${sample#*/}
    count=$(grep ">" $f | wc -l)
    echo $sample_2,$count
done >> phage_count.csv
# Total metagenome samples
for f in total_out/*summary/*virus.fna
do
    sample=${f%%_summary*}
    sample_2=${sample#*/}
    count=$(grep ">" $f | wc -l)
    echo $sample_2,$count
done >> phage_count.csv

In [ ]:
# copy phage seqs into a new file
## all total metagenome phage
cd /home/dsbard/bee_phage/reviewer_approach/genomad_out/viral_seqs/total
cat *.fna >> all_total_phage.fna
# rename them total_0-X
conda activate bbduk
rename.sh in=all_total_phage.fna \
out=all_total_phage_renamed.fna  \
prefix=total_phage_
grep ">" all_total_phage_renamed.fna | wc -l
# 938 phage found
cd /home/dsbard/bee_phage/reviewer_approach/genomad_out/viral_seqs/virome
cat *.fna >> all_virome_phage.fna
rename.sh in=all_virome_phage.fna \
out=all_virome_phage_renamed.fna  \
prefix=virome_phage_
grep ">" all_virome_phage_renamed.fna | wc -l
# 2915 phage found

# Reviewers also suggested we use drep instead of cdhit (as we had done previously) and that we use checkV to remove low quality phage seqs
# now combine virome and total metagenome phage in a new file for dereplication
cd /home/dsbard/bee_phage/reviewer_approach/
mkdir drep
cd drep
mkdir log viral_seqs
# write new file
s

# Run checkV
#!/bin/bash
#SBATCH --job-name=checkV
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --ntasks=16
#SBATCH --output=/home/dsbard/bee_phage/reviewer_approach/checkv/log/%j.out
#SBATCH --partition=bigmemm
#SBATCH --mem=120GB

conda activate checkv

checkv end_to_end \
/home/dsbard/bee_phage/reviewer_approach/checkv/all_phage_seqs.fna \
/home/dsbard/bee_phage/reviewer_approach/checkv/out \
-d /home/dsbard/checkv/checkv-db-v1.5 \
-t 16

mkdir trimmed
cat out/*.fna >> trimmed/trimmed_seqs.fa

# Filter to retain only high quality viral seqs
# (this is done in R)
# 655 ~high quality~ phage left

sed 's/_1 .*$//' trimmed_seqs.fa > cleaned_trimmed_seqs.fa
# now lets split em
cd /home/dsbard/bee_phage/reviewer_approach/checkv/trimmed
cat cleaned_trimmed_seqs.fa | awk '{
        if (substr($0, 1, 1)==">") {filename=(substr($0,2) ".fa")}
        print $0 > filename
}'
mv virome_phage* seqs/
mv total* seqs/

# use list of good phage to pull high quality phage genomes
while read line
do 
cp trimmed/seqs/${line}.fa \
/home/dsbard/bee_phage/reviewer_approach/drep/good_seqs
done < good_viral_seqs.csv

# run drep
#!/bin/bash
#SBATCH --job-name=drep
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --ntasks=12
#SBATCH --output=/home/dsbard/bee_phage/reviewer_approach/drep/log/%j.out
#SBATCH --partition=bigmemm
#SBATCH --mem=120GB


conda activate drep

dRep dereplicate /home/dsbard/bee_phage/reviewer_approach/drep/dereplicated_phage \
-g /home/dsbard/bee_phage/reviewer_approach/drep/good_seqs/*.fa \
-l 5000 \
-nc .85 \
--ignoreGenomeQuality

cd /home/dsbard/bee_phage/reviewer_approach/drep/dereplicated_phage/dereplicated_genomes
ls -1 | wc -l
# 609 phage out
# 655 phage in

In [ ]:
# Calculate coverage

#!/bin/bash
#SBATCH --job-name=cover_%j
#SBATCH --nodes=1
#SBATCH -t 6:00:00 
#SBATCH --ntasks=12
#SBATCH --output=/home/dsbard/coverm/log/cover%j.out
#SBATCH --partition=med

# module load deprecated/coverm/0.6.1  
# source activate coverm-0.6.1
# module load samtools/1.14 
source /home/dsbard/miniconda3/bin/activate
conda activate /group/nas-10-1-export-4/dsbard/yes/envs/coverm

coverm genome -t 12 \
-1 /group/nas-10-1-export-4/dsbard/bee_phage/dls/hybrid/decontam/$1 \
-2 /group/nas-10-1-export-4/dsbard/bee_phage/dls/hybrid/decontam/${1%%R1*}R2.fastq.gz \
--genome-fasta-directory /group/nas-10-1-export-4/dsbard/bee_phage/reviewer_approach/drep/dereplicated_phage/dereplicated_genomes/ \
--genome-fasta-extension .fa \
--min-read-percent-identity 95 \
--min-covered-fraction=75 \
--min-read-aligned-percent 75 \
-m trimmed_mean \
-m covered_fraction \
--output-file /home/dsbard/coverm/out/${1%%_host*}_cover_result.tsv




